In [1]:
pip install requests


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import json

relation_id = 14552265
overpass_url = "http://overpass-api.de/api/interpreter"

In [3]:
query = f"""
[out:json][timeout:25];
relation({relation_id});
(._; >>;);
out body;
"""
response = requests.post(overpass_url, data={"data": query})
data = response.json()

node_dict = {}
ways = []

# Code 1 ( lấy tòa độ với ko có độ cao )

In [4]:
# # Thu thập node và way
# for element in data["elements"]:
#     if element["type"] == "node":
#         node_dict[element["id"]] = (element["lat"], element["lon"])
#     elif element["type"] == "way":
#         ways.append(element)

# # Kết quả đầu ra dạng danh sách các polygon
# polygons = []

# for way in ways:
#     coords = []
#     for node_id in way["nodes"]:
#         if node_id in node_dict:
#             lat, lon = node_dict[node_id]
#             coords.append([lon, lat])  # chuyển sang [lon, lat]

#     # Đảm bảo polygon được đóng kín
#     if coords and coords[0] != coords[-1]:
#         coords.append(coords[0])

#     if coords:
#         polygon = {
#             "type": "polygon",
#             "rings": coords,
#             "symbol": {
#                 "type": "simple-fill",
#                 "color": [227, 139, 79, 0.6],
#                 "outline": {
#                     "color": [255, 255, 255],
#                     "width": 1
#                 }
#             }
#         }
#         polygons.append(polygon)

# # Xuất kết quả
# print(json.dumps(polygons, indent=2))


# Code 2 ( lấy tòa độ có độ cao )

In [5]:
import json

# Giữ nguyên phần xử lý dữ liệu ban đầu...
for element in data["elements"]:
    if element["type"] == "node":
        node_dict[element["id"]] = (element["lat"], element["lon"])
    elif element["type"] == "way":
        ways.append(element)

features = []

for way in ways:
    height_val = None
    if "tags" in way:
        tags = way["tags"]
        if "height" in tags:
            try:
                height_val = float(tags["height"].replace("m", ""))
            except:
                height_val = None
        elif "building:levels" in tags:
            try:
                height_val = float(tags["building:levels"]) * 3
            except:
                height_val = None

    if height_val is None:
        height_val = 10.0

    coords = []
    for node_id in way["nodes"]:
        if node_id in node_dict:
            lat, lon = node_dict[node_id]
            lon = round(lon, 14)
            lat = round(lat, 14)
            coords.append([lon, lat, height_val])

    if coords and coords[0] != coords[-1]:
        coords.append(coords[0])

    if coords:
        feature = {
            "type": "Feature",
            "properties": {
                "Building name": way["tags"].get("name", "Unknown Building") if "tags" in way else "Unknown Building",
                "height": height_val - 7.8,
                "idb": str(way["id"])
            },
            "geometry": {
                "type": "Polygon",
                "coordinates": [coords]
            },
            "id": f"pp{way['id']}"
        }
        features.append(feature)

# Mỗi feature là một FeatureCollection riêng biệt
for feature in features:
    single_feature_collection = {
        "type": "FeatureCollection",
        "generator": "smartcity",
        "copyright": "Smartcity",
        "timestamp": "2025-04-18T00:00:00Z",
        "features": [feature]
    }
    print(json.dumps(single_feature_collection, indent=2)+",")



{
  "type": "FeatureCollection",
  "generator": "smartcity",
  "copyright": "Smartcity",
  "timestamp": "2025-04-18T00:00:00Z",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "Building name": "L\u0103ng Ch\u1ee7 t\u1ecbch H\u1ed3 Ch\u00ed Minh",
        "height": 0.0,
        "idb": "37625751"
      },
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              105.8348121,
              21.036909,
              7.8
            ],
            [
              105.8348196,
              21.036944,
              7.8
            ],
            [
              105.8348983,
              21.0369294,
              7.8
            ],
            [
              105.8348595,
              21.0367475,
              7.8
            ],
            [
              105.8348205,
              21.0365651,
              7.8
            ],
            [
              105.8347467,
              21.0365789,
              7.8